In [1]:
%run ../prepare_data.py -N_users 1

D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
sequence = transform_users(
    users.user_id.sample(1000)
)
df_train = sequence

print(df_train.shape)

D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(191902, 82)


In [9]:
sequence = transform_users(
    users.user_id.sample(200)
)
df_test = sequence

print(df_test.shape)

D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Users\greg_\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(35802, 82)


In [12]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.optimizers import RMSprop, adam
from keras.utils.data_utils import get_file
from keras.callbacks import History, ModelCheckpoint
import numpy as np
import pandas as pd
import random
import sys
import string
import matplotlib.pyplot as plt
%matplotlib inline


# Define number of inputs/outputs to handle
input_size = 81
output_size= 16

# Load the train data
...

print('Train total outputs:', output_size)

# Pre-process the data (normalize)

# cut the input in semi-redundant sequences of maxlen characters
maxlen = 90
step_days = 3



def reformat(df_in):
    max_sequences = 10**5
    days_sequence = np.empty((max_sequences,maxlen,input_size),dtype=int)
    next_day = np.empty((max_sequences,output_size),dtype=int)
    days = df_in[:,-1]
    df = df_in[:,:-1]
    
    j=0
    last_day = 0
    for day_i in range(0, df.shape[0] - maxlen, step_days):
        if last_day < days[day_i+maxlen]:
            days_sequence[j] = df[day_i: day_i + maxlen,:input_size]
        next_day[j] = df[day_i + maxlen,:output_size]
        j += 1
        last_day = days[day_i]

    days_sequence = days_sequence[:j,:,:]
    next_day = next_day[:j,:]
    
    return days_sequence,next_day

X_train, y_train = reformat(df_train)
X_test,y_test=reformat(df_test)

print(df_train.shape,df_test.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Train total outputs: 16
(191902, 82) (35802, 82)
(63938, 90, 81)
(63938, 16)
(11904, 90, 81)
(11904, 16)


In [13]:
def plot_logs(history):
    """
    Plot the accuracy and loss for 
        training and test sets
    """
    evaluation_cost = history.history['val_loss']
    evaluation_accuracy = history.history['val_acc']
    training_cost = history.history['loss']
    training_accuracy = history.history['acc']
    f, (ax1, ax2) = plt.subplots(1, 2)
    f.set_figwidth(10)
    ax1.plot(evaluation_cost,label= 'test')
    ax1.plot(training_cost, label='train')
    ax1.set_title('Cost')
    ax1.legend()
    ax2.plot(evaluation_accuracy, label='test')
    ax2.plot(training_accuracy, label='train')
    ax2.set_title('Accuracy')
    ax2.legend(loc='lower right')


def sample(preds, temperature=1.0):
    """
    Generate the next sequence
    Low temperature means very conservative (picks more probable most of the time)
    High temperature means very adventurous (picks less probable more frequently)
    """
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_prediction(history,days=28, maxlen=60, input_size=16, output_size=16, diversity=1):
    """
    Generates as many days of prediction as requested
    Considers maxlen days of past history (must be aligned with model)
    """
    generated = np.zeros((days,output_size))
    if history.shape[1]>maxlen:
        x = history[:,-maxlen-1:-1,:input_size]
    else:
        x = history[:,:,:input_size]
    #print(x.shape)
    for i in range(days):
        #print("Day %d" % i)
        preds = model.predict(x, verbose=0)[0].reshape(output_size)
        #print(preds)
        generated[i,:] = preds
        
        if input_size > output_size:
            res = np.zeros(input_size)
            res[:output_size] = preds
            preds = res

        #print(preds.shape)
        #next_symptoms = sample(preds, diversity)
        next_symptoms = preds
        #print(next_symptoms)


        x[:,:maxlen-1,:] = x[:,1:,:]
        x[:,maxlen-1,:] = next_symptoms

    return generated

In [ ]:
# Model params
batch_size = 256
nb_epoch = 500
np.random.seed(131078)

# build the model
print('Build model...')

"""
filepath="lstm_1_layer.hdf5"
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, input_size)))
model.add(Dense(output_size))
model.add(Activation('sigmoid'))

"""
filepath="lstm_2_layers_higher_dropout.hdf5"
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, input_size), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(256))
model.add(Dropout(0.7))
model.add(Dense(output_size))
model.add(Activation('sigmoid'))


optimizer = adam()
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer,
              metrics=['accuracy'])

# Define callback to save model
save_snapshots = ModelCheckpoint(filepath,
                                 monitor='loss',
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='min',
                                 verbose=0)
callbacks_list = [save_snapshots]

history = model.fit(X_train,
                    y_train,
                    batch_size=batch_size,
                    nb_epoch=nb_epoch,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=1)          

score = model.evaluate(X_test, y_test, verbose=2)
print('Test score:', score[0])
print('Test accuracy:', score[1])  

plot_logs(history)          


Build model...
Train on 63938 samples, validate on 11904 samples
Epoch 1/500
63938/63938 [==============================] - 127s - loss: 0.1061 - acc: 0.9778 - val_loss: 0.0832 - val_acc: 0.9830
Epoch 2/500
63938/63938 [==============================] - 124s - loss: 0.0823 - acc: 0.9837 - val_loss: 0.0806 - val_acc: 0.9830
Epoch 3/500
63938/63938 [==============================] - 124s - loss: 0.0791 - acc: 0.9837 - val_loss: 0.0777 - val_acc: 0.9830
Epoch 4/500
63938/63938 [==============================] - 124s - loss: 0.0777 - acc: 0.9837 - val_loss: 0.0772 - val_acc: 0.9830
Epoch 5/500
63938/63938 [==============================] - 123s - loss: 0.0769 - acc: 0.9837 - val_loss: 0.0771 - val_acc: 0.9830
Epoch 6/500
63938/63938 [==============================] - 124s - loss: 0.0772 - acc: 0.9837 - val_loss: 0.0771 - val_acc: 0.9830
Epoch 7/500
63938/63938 [==============================] - 124s - loss: 0.0765 - acc: 0.9837 - val_loss: 0.0762 - val_acc: 0.9830
Epoch 8/500
63938/63938 [

In [49]:
hist = X_test[0,:,:].reshape(1,maxlen,-1)
print(hist.shape)
res = generate_prediction(hist,maxlen=maxlen,input_size=input_size,output_size=output_size)
res.shape

(1, 10, 81)


(28, 16)

In [50]:
res[:,:output_size]

array([[ 0.01095976,  0.00664205,  0.00363475,  0.01084435,  0.00173918,
         0.00605508,  0.00538483,  0.01100782,  0.0215272 ,  0.00720003,
         0.00308284,  0.00900103,  0.00516408,  0.00340249,  0.00277895,
         0.00186608],
       [ 0.01095976,  0.00664205,  0.00363475,  0.01084435,  0.00173918,
         0.00605508,  0.00538483,  0.01100782,  0.0215272 ,  0.00720003,
         0.00308284,  0.00900103,  0.00516408,  0.00340249,  0.00277895,
         0.00186608],
       [ 0.01095976,  0.00664205,  0.00363475,  0.01084435,  0.00173918,
         0.00605508,  0.00538483,  0.01100782,  0.0215272 ,  0.00720003,
         0.00308284,  0.00900103,  0.00516408,  0.00340249,  0.00277895,
         0.00186608],
       [ 0.01095976,  0.00664205,  0.00363475,  0.01084435,  0.00173918,
         0.00605508,  0.00538483,  0.01100782,  0.0215272 ,  0.00720003,
         0.00308284,  0.00900103,  0.00516408,  0.00340249,  0.00277895,
         0.00186608],
       [ 0.01095976,  0.00664205,  0

In [51]:
y_test[:28,:output_size]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def format_prediction(prediction,user_id):
    output = []
    for i,row in enumerate(prediction):
        for j, symptom in enumerate(row):
            line = user_id + "," + i + "," + j + "," + prediction[i,j]
            ouput.append(line)

def pad_reshape_history(sequence,maxlen):
    if sequence.shape[0] < maxlen:
        hist = np.zeros((maxlen,sequence.shape[1]))
        hist[maxlen-sequence.shape[0]:,:0] = sequence
    else:
        hist = sequence[-maxlen-1:-1,:]
    hist = hist.reshape(1,maxlen,-1)

for woman in cycle0:
    current_id = woman.user_id
    expected_length = np.ceil(woman.expected_cycle_length)
    sequence = transform_users(current_id)
    hist = pad_reshape_history(sequence,maxlen)
    print(hist.shape)
    res = generate_prediction(hist,maxlen=maxlen,input_size=input_size,output_size=output_size,days=expected_length)
    print(res.shape)
    
    